In [4]:
#import data
import json
import codecs

with codecs.open('poems.json') as f:
    data = json.load(f)

In [5]:
# number of data points in each category
print [(cate, len(l)) for cate, l in data.items()]

[(u'songbie', 37), (u'tianyuan', 23), (u'shanshui', 53), (u'zhanzheng', 54), (u'aiqing', 41)]


In [6]:
# find the set of unique characters
all_poems = sum(data.values(), [])
alphabet = "".join(all_poems)

In [7]:
alphabet = alphabet.replace(u"。", "")
alphabet = alphabet.replace(u"，", "")
alphabet = alphabet.replace(u"！", "")
alphabet = alphabet.replace(u"？", "")

In [8]:
# number of characters
n_alp = len(set(alphabet))
print n_alp

2170


In [9]:
# mapping of characters and number codes
alphabet = list(set(alphabet))
index = range(len(alphabet))
encoding_dict = dict(zip(alphabet, index))
decoding_dict = dict(zip(index, alphabet))

In [10]:
poem_sample = data.values()[0][0]

In [11]:
# turn a poem into clauses (separated by punctuation marks)
def poem_to_clauses(poem):
    temp = poem.replace(u"，", u"。")
    temp = temp.replace(u"！", u"。")
    temp = temp.replace(u"？", u"。")
    clauses = temp.split(u"。")
    if len(clauses[-1]) <= 1:
        return clauses[:-1]
    else:
        return clauses

In [12]:
# turn a clause to a list of number codes
def clause_to_code(clause):
    return [encoding_dict[_] for _ in clause]
temp = poem_to_clauses(poem_sample)[0]
print clause_to_code(temp)

[1232, 922, 692, 832, 1409, 13, 1769]


In [13]:
# encode all data
import numpy as np
def encode_data(data):
    X = {}
    for cate, poem_list in data.items():
        X[cate] = []
        for poem in poem_list:
            clauses = poem_to_clauses(poem)
            coded_clauses = [clause_to_code(_) for _ in clauses]
            X[cate].append(coded_clauses)
    return X

# decode functions
def code_to_clause(clause):
    return [decoding_dict[_] for _ in clause]

def code_to_poem(clauses):
    lines = []
    for clause in clauses:
        line = "".join(code_to_clause(clause))
        lines.append(line)
        
    return u"，".join(lines)+u"。"
        
X = encode_data(data)

In [14]:
# training markov models
transition = {}
initial = {}
for cate, poem_list in X.items():
    A = np.zeros((n_alp, n_alp))
    a = np.zeros((n_alp))
    for poem in poem_list:
        for clause in poem:
            a[clause[0]] += 1
            for i in range(len(clause)-1):
                A[clause[i+1], clause[i]] += 1
    A_sum = A.sum(axis=0)
    A_sum[A_sum==0] = 1
    A = A / A_sum
    a /= a.sum()
    transition[cate] = A
    initial[cate] = a

In [15]:
categories = X.keys()

In [51]:
# side goal: generate fake poems
import matplotlib.pyplot as plt
from collections import Counter
def generate(poems, initial, transition):
    lens = [len(poem) for poem in poems]
    counts = Counter(lens)
    values, occur = zip(*counts.items())
    occur = np.array(occur).astype(float)
    occur /= float(occur.sum())
    length = np.random.choice(values, p=occur)
    
    n_chars = [len(poem[0]) for poem in poems]
    counts = Counter(n_chars)
    values, occur = zip(*counts.items())
    occur = np.array(occur).astype(float)
    occur /= float(occur.sum())
    len_per_line = np.random.choice(values, p=occur)
    
    poem = []
    for i in range(length):
        clause = []
        clause.append(np.random.choice(range(2170), p=initial))
        for j in range(len_per_line-1):
            if transition[:,clause[-1]].sum()>0:
                clause.append(np.random.choice(range(2170), p=transition[:, clause[-1]]))
            else:
                clause.append(np.random.choice(range(2170)))
        poem.append(clause)
    print code_to_poem(poem)

for cate in categories:
    print cate
    for i in range(10):
        generate(X[cate],initial[cate],transition[cate])
        print

songbie
山路分处报故人，岂能愁云西风知，风引雨连城边古，挥手泪空相向晚。

劳劳劳劳送马川，草黄鹤楼船扫地，初渡河桃花三月，瀚海楼船扫地白。

青枫江入吴步样，边头人讼芋田裳，山中军角声向临，日暮掩柴扉试惯。

升沉应为客散暮，文翁翻嫌易磋砣，不尽水急流水阔，幕中一曲解行泪。

青夹御苑砧声向，挥手自兹去时分，与羌笛桑佣手泪，山六千尺沙莽黄，山一丈夫孩鞞浙，火山陲楠广井奋，相拨颈宇粤樽感，江边古木疏掳屋。

随风尘飞鸟，嗟君迟楼船，忽闻游子唱，月向人归雁。

火山陲彼似春流，火山此别燕丹认，云山独归远芳树，不觉有离歌声一，不暖锦衾薄估房，铙吹又送巴人已。

王孙去莫复，青青枫江风，还从此地别，文翁翻教授，唯有离伤心，天际流绕蜀，长安行处期，崎岖不得意。

望君离魂倍，好去疾如刀，山随平野火，鸿雁不足论，今日五津亭，山独归客稀，周瑜于此地，青山此地石。

二龙争战决雌雄，翻教授拨崔离歌，岂能愁里辞家事，料知别苦粱樵浓，赤亭道同是茫客，细碎石大如相向，青山响杜鹃尸缨，火山六月夜寒云。

tianyuan
惟先生万户，坐看新历剡，官府鸣一曲，长松下孤烟，明田家事南，故巢鸣珂有，此情束鸦初，缀景未及郊。

征车自来过，积雨里南亩，三径慰竟他，村流斧伐远，南邻近石出，吴钩近石火，再见封侯万，晨兴来就菊。

中庭栀子敲，饥小尾扑花，桥斜阳照渔，松风急蛾正。

望断金马门，松下清江村，柳先生万户，故人别业水，遥岚破烟阁，开朝槿舰娆，木啭黄桑麻，苗秀眉老与。

膏释原野老念牧，舍南园林叟坼阵，野老去自课越佣，丘园露葵朝归村，柳花源里缲丝挑，自念牧童未已暮，踏雪浦运寄成远，缀景常晏反岩旄，松风寺外斜阳照，绿波春水新泉脉。

童未已暮锄，宫帘挂玉弓，蒸藜炊黍归，苍苍横翠微。

待到重客犹，饥小甲蔬策，鱼拥香琥珀，村路元无雨，即此中鸥何，园多怀镇邀，轻绡一双苍，望断金马牙，天边合嫩绿，无知己样药，微躯此情软，嫁与春草远，曼倩诙谐取，收橡实榆柚。

天羡鸿鹄戏骧流，遥岚破月随人看，乡曲卧鸳鸯暖诤，丘园日斜阳照墟，明田父对门临觞，故人看新晴原野，蒸藜炊黍归来径，白长歌吟松下孤。

故池想芜没会，蚕眠桑叶稀袅，三十州胡燕识，野童稚开且为，白长峦谷口倚，郭门万户侯万，天边渔船中习，黄桑麻荣近石。

南亩当榛荆，野童未遇压，花惊雪也相，晨兴来堂上，竹入幽鸟笑，绿波春候荆，云光岚彩四，耕者黛

In [16]:
transition["shanshui"][:,1].sum()

1.0

In [17]:
# finding smallest nonzero entries
for cate, sub in initial.items():
    print np.min(sub[sub>0])

0.00408163265306
0.00380228136882
0.0018691588785
0.00104821802935
0.00212314225053


In [18]:
0.0625
0.0909090909091
0.0185185185185
0.0204081632653
0.03125
0.00408163265306
0.00380228136882
0.0018691588785
0.00104821802935
0.00212314225053

0.00212314225053

In [19]:
# prediction function

import operator
def log_p(p):
    if p == 0:
        return -8
    else:
        return np.log(p)


def predict(poem, initial, transition):
    log_proba = {}
    for cate in categories:
        p = 0
        for clause in poem:
            p += log_p(initial[cate][clause[0]])
            for i in range(len(clause)-1):
                p += log_p(transition[cate][clause[i+1], clause[i]])
        log_proba[cate] = p
    return max(log_proba.iteritems(), key=operator.itemgetter(1))[0]

#print predict(X["tianyuan"][2])

In [20]:
n_by_cate =  dict([(cate, len(l)) for cate, l in data.items()])

In [21]:
categories

[u'songbie', u'tianyuan', u'shanshui', u'zhanzheng', u'aiqing']

In [22]:
cate_num = dict(zip(categories, range(5)))

In [23]:
# LOOCV with unbalanced data

def LOO(X):
    set_aside_index = {cate: np.random.randint(0, n) for cate, n in n_by_cate.items()}
    
    transition = {}
    initial = {}
    for cate, poem_list in X.items():
        A = np.zeros((n_alp, n_alp))
        a = np.zeros((n_alp))
        for poem_id, poem in enumerate(poem_list):
            if poem_id != set_aside_index[cate]:
                for clause in poem:
                    a[clause[0]] += 1
                    for i in range(len(clause)-1):
                        A[clause[i+1], clause[i]] += 1
        A_sum = A.sum(axis=0)
        A_sum[A_sum==0] = 1
        A = A / A_sum
        a /= a.sum()
        transition[cate] = A
        initial[cate] = a
    
    predicted = {}
    for cate, i in set_aside_index.items():
        predicted[cate] = predict(X[cate][i], initial, transition)
    return predicted

def LOOCV(n_trials=1000):
    acc_by_cate = {cate:0 for cate in n_by_cate}
    confusion = np.zeros((5,5))
    for i in range(n_trials):
        predicted = LOO(X)
        for cate, result in predicted.items():
            confusion[cate_num[result], cate_num[cate]]+=1
            if cate == result:
                acc_by_cate[cate] += 1
                
        
    print acc_by_cate
    print sum(acc_by_cate.values())/(5.*n_trials)
    return acc_by_cate, confusion
    
acc, confusion0 = LOOCV()

{u'songbie': 306, u'tianyuan': 400, u'shanshui': 572, u'zhanzheng': 898, u'aiqing': 412}
0.5176


In [24]:
# confusion matrix
confusion0 /= confusion0.sum()

In [25]:
confusion0 *= 100

In [26]:
confusion0

array([[  6.12,   0.74,   1.74,   1.3 ,   1.12],
       [  0.04,   8.  ,   0.64,   0.04,   0.04],
       [  6.44,   4.72,  11.44,   0.42,   5.52],
       [  5.58,   6.48,   2.4 ,  17.96,   5.08],
       [  1.82,   0.06,   3.78,   0.28,   8.24]])

In [27]:
confusion0.sum(axis=1)

array([ 11.02,   8.76,  28.54,  37.5 ,  14.18])

In [28]:
# LOOCV with balanced data
def LOO1(X):
    set_aside_index = {cate: np.random.randint(0, n) for cate, n in n_by_cate.items()}
    
    transition = {}
    initial = {}
    for cate, poem_list in X.items():
        A = np.zeros((n_alp, n_alp))
        a = np.zeros((n_alp))
        count = 0
        for poem_id, poem in enumerate(poem_list):
            if poem_id != set_aside_index[cate]:
                for clause in poem:
                    a[clause[0]] += 1
                    for i in range(len(clause)-1):
                        A[clause[i+1], clause[i]] += 1
                count += 1
            if count>=22: break            
        A_sum = A.sum(axis=0)
        A_sum[A_sum==0] = 1
        A = A / A_sum
        a /= a.sum()
        transition[cate] = A
        initial[cate] = a
    
    predicted = {}
    for cate, i in set_aside_index.items():
        predicted[cate] = predict(X[cate][i], initial, transition)
    return predicted
print LOO1(X)

{u'songbie': u'shanshui', u'tianyuan': u'tianyuan', u'shanshui': u'aiqing', u'zhanzheng': u'zhanzheng', u'aiqing': u'zhanzheng'}


In [29]:
def LOOCV1(n_trials=1000):
    acc_by_cate = {cate:0 for cate in n_by_cate}
    confusion = np.zeros((5,5))
    for i in range(n_trials):
        predicted = LOO1(X)
        for cate, result in predicted.items():
            confusion[cate_num[result], cate_num[cate]]+=1
            if cate == result:
                acc_by_cate[cate] += 1
            
                
        
    print acc_by_cate
    print sum(acc_by_cate.values())/(5.*n_trials)
    return acc_by_cate, confusion
acc1, confusion1 = LOOCV1()

{u'songbie': 303, u'tianyuan': 572, u'shanshui': 374, u'zhanzheng': 851, u'aiqing': 453}
0.5106


In [30]:
confusion1 /= 5000.

In [31]:
confusion1 *= 100

In [33]:
confusion1

array([[  6.06,   1.28,   3.84,   0.62,   0.84],
       [  1.5 ,  11.44,   2.54,   0.26,   2.72],
       [  5.66,   3.02,   7.48,   0.84,   4.12],
       [  4.52,   3.06,   2.24,  17.02,   3.26],
       [  2.26,   1.2 ,   3.9 ,   1.26,   9.06]])

In [32]:
confusion1.sum(axis=1)

array([ 12.64,  18.46,  21.12,  30.1 ,  17.68])